In [21]:
import tensorflow
print(tensorflow.__version__)

2.8.2


In [22]:
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt
import numpy as np


# Data processing

In [23]:
(xtn, ytn), (xtt, ytt) = K.datasets.cifar10.load_data()
print(xtn.shape, ytn.shape, xtt.shape, ytt.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [24]:
def preprocess_data(X, Y):
  x = K.applications.densenet.preprocess_input(X)
  y = K.utils.to_categorical(Y, 10)
  return x, y

In [26]:
xtn, ytn = preprocess_data(xtn, ytn)
xtt, ytt = preprocess_data(xtt, ytt)

print(xtn[0].shape, ytn[0].shape)

(32, 32, 3) (10,)


# Transfer Learning

In [27]:
base_model = K.applications.DenseNet121(include_top=False,
                                        weights='imagenet',
                                        pooling='avg',
                                        input_shape=(224, 224, 3))
base_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_4 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_6[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_4[0][0]']       
                                )                                                       

In [29]:
inputs = K.Input(shape=(32, 32, 3))
input = K.layers.Lambda(lambda image : tf.image.resize(image, (224, 224)))(inputs)

x = base_model(input, training=False)

# x = K.layers.GlobalAveragePooling2D()(x)

x = K.layers.Dense(500, activation='relu')(x)
x = K.layers.Dropout(0.3)(x)
outputs = K.layers.Dense(10, activation='softmax')(x)

model = K.Model(inputs, outputs)

# Model with no trainable

In [30]:
base_model.trainable = False

optimizer = K.optimizers.Adam()

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lambda_4 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 1024)              7037504   
                                                                 
 dense_6 (Dense)             (None, 500)               512500    
                                                                 
 dropout_3 (Dropout)         (None, 500)               0         
                                                                 
 dense_7 (Dense)             (None, 10)                5010      
                                                                 
Total params: 7,555,014
Trainable params: 517,510
Non-train

In [31]:
history = model.fit(xtn,
                    ytn,
                    validation_data=(xtt, ytt),
                    batch_size=300,
                    epochs=4,
                    verbose=1)
model.save('/content/drive/MyDrive/Colab Notebooks/supervised_learning/tests for 0x09/cifar10_DenseN121.h5')

Epoch 1/4
167/167 [==============================] - 196s 1s/step - loss: 0.5329 - acc: 0.8193 - val_loss: 0.3466 - val_acc: 0.8827
Epoch 2/4
167/167 [==============================] - 162s 972ms/step - loss: 0.3400 - acc: 0.8823 - val_loss: 0.3160 - val_acc: 0.8959
Epoch 3/4
167/167 [==============================] - 162s 973ms/step - loss: 0.3019 - acc: 0.8955 - val_loss: 0.3066 - val_acc: 0.8955
Epoch 4/4
167/167 [==============================] - 162s 972ms/step - loss: 0.2786 - acc: 0.9046 - val_loss: 0.2950 - val_acc: 0.9003


# Fine tunning

In [32]:
from tensorflow.keras.utils import plot_model
plot_model(base_model, to_file='/content/drive/MyDrive/Colab Notebooks/supervised_learning/tests for 0x09/model_Dens121.png')

Output hidden; open in https://colab.research.google.com to view.

In [34]:
for i, layer in enumerate(base_model.layers):
  print(i, '-', layer.name)

0 - input_6
1 - zero_padding2d_4
2 - conv1/conv
3 - conv1/bn
4 - conv1/relu
5 - zero_padding2d_5
6 - pool1
7 - conv2_block1_0_bn
8 - conv2_block1_0_relu
9 - conv2_block1_1_conv
10 - conv2_block1_1_bn
11 - conv2_block1_1_relu
12 - conv2_block1_2_conv
13 - conv2_block1_concat
14 - conv2_block2_0_bn
15 - conv2_block2_0_relu
16 - conv2_block2_1_conv
17 - conv2_block2_1_bn
18 - conv2_block2_1_relu
19 - conv2_block2_2_conv
20 - conv2_block2_concat
21 - conv2_block3_0_bn
22 - conv2_block3_0_relu
23 - conv2_block3_1_conv
24 - conv2_block3_1_bn
25 - conv2_block3_1_relu
26 - conv2_block3_2_conv
27 - conv2_block3_concat
28 - conv2_block4_0_bn
29 - conv2_block4_0_relu
30 - conv2_block4_1_conv
31 - conv2_block4_1_bn
32 - conv2_block4_1_relu
33 - conv2_block4_2_conv
34 - conv2_block4_concat
35 - conv2_block5_0_bn
36 - conv2_block5_0_relu
37 - conv2_block5_1_conv
38 - conv2_block5_1_bn
39 - conv2_block5_1_relu
40 - conv2_block5_2_conv
41 - conv2_block5_concat
42 - conv2_block6_0_bn
43 - conv2_block6_

In [35]:
for layer in base_model.layers[:141]:
  layer.trainable=False
for layer in base_model.layers[141:]:
  layer.trainable=True

In [36]:
optimizer = K.optimizers.Adam(1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])
history = model.fit(xtn,
                    ytn,
                    validation_data=(xtt, ytt),
                    batch_size=300,
                    epochs=4,
                    verbose=1)
model.save('/content/drive/MyDrive/Colab Notebooks/supervised_learning/tests for 0x09/cifar10_DenseN121.h5')

Epoch 1/4
167/167 [==============================] - 168s 977ms/step - loss: 0.2407 - acc: 0.9162 - val_loss: 0.2880 - val_acc: 0.9016
Epoch 2/4
167/167 [==============================] - 162s 973ms/step - loss: 0.2364 - acc: 0.9187 - val_loss: 0.2866 - val_acc: 0.9028
Epoch 3/4
167/167 [==============================] - 162s 971ms/step - loss: 0.2342 - acc: 0.9194 - val_loss: 0.2853 - val_acc: 0.9026
Epoch 4/4
167/167 [==============================] - 162s 971ms/step - loss: 0.2311 - acc: 0.9198 - val_loss: 0.2847 - val_acc: 0.9030
